In [1]:
"""Verify the gradient derivation of linear masked autoencoder."""
import torch
import torch.nn as nn

In [2]:
# Initialise matrix and vector
m = 50
n = 10
prob = 0.6
X = torch.rand(m, n)
W1 = torch.rand(n, n, requires_grad=True)
W2 = torch.rand(n, n, requires_grad=True)

In [3]:
# Define different types of masks
def mask_basic(prob, m, n):
    return torch.zeros(m, n).bernoulli_(prob)

def mask_dropping_probs(prob_list: torch.Tensor, m, n):
    return torch.zeros(m, n).bernoulli_(prob_list)

def mask_patches(prob, patch_size, m, n):
    if not n % patch_size:
        pix_num = n // patch_size
        mat_patches = torch.zeros(m, pix_num).bernoulli_(prob)
        return mat_patches.repeat_interleave(patch_size, dim=1)
    else:
        NotImplementedError

In [4]:
# define loss function in terms of W1 and W2
def loss_func_W1_basic(W1):
    z = (mask_basic(prob, m, n)*X) @ W1.T @ W2.T - X
    return sum(sum(z*z)) / m / n

def loss_func_W2_basic(W2):
    z = (mask_basic(prob, m, n)*X) @ W1.T @ W2.T - X
    return sum(sum(z*z)) / m / n

In [5]:
# find the theoretical and numerical solutions of W1 and W2
# mask_basic
mean_m_basic = torch.ones(m, n) * prob
square_m_basic = torch.ones(n, n) * prob**2
square_m_basic.fill_diagonal_(prob)

grad_w1_theory_basic = W2.T @ (W2@W1@(square_m_basic*(X.T@X)) - X.T@(mean_m_basic*X)) * (2/m/n)
grad_w2_theory_basic = (W2@W1@(square_m_basic*(X.T@X))-X.T@(mean_m_basic*X)) @ W1.T * (2/m/n)

# Initialising sampling
N = 10000
grad_w1_numer_basic = 0
grad_w2_numer_basic = 0
# Sampling process
for i in range(N):
    grad_w1_numer_basic += torch.autograd.functional.jacobian(loss_func_W1_basic, W1)
    grad_w2_numer_basic += torch.autograd.functional.jacobian(loss_func_W2_basic, W2)
    #W1.detach()
    #W2.detach()
grad_w1_numer_basic /= N
grad_w2_numer_basic /= N

abs(grad_w1_theory_basic-grad_w1_numer_basic)

tensor([[2.6019e-03, 2.9137e-03, 4.4494e-03, 2.5742e-03, 1.6794e-03, 1.5836e-03,
         1.1778e-04, 6.8595e-03, 4.6158e-03, 3.0077e-03],
        [2.7554e-03, 3.0451e-03, 4.6487e-03, 2.6863e-03, 1.7796e-03, 1.6692e-03,
         1.0705e-04, 7.2539e-03, 4.8516e-03, 3.1261e-03],
        [1.7192e-03, 1.9151e-03, 2.9219e-03, 1.6539e-03, 1.1305e-03, 1.0265e-03,
         8.7976e-05, 4.5731e-03, 3.0341e-03, 1.9590e-03],
        [2.1687e-03, 2.3961e-03, 3.7491e-03, 2.2495e-03, 1.4198e-03, 1.3301e-03,
         9.1553e-05, 5.7590e-03, 3.9306e-03, 2.5003e-03],
        [1.9863e-03, 2.1677e-03, 3.4163e-03, 2.0180e-03, 1.2987e-03, 1.2164e-03,
         9.4652e-05, 5.2874e-03, 3.5596e-03, 2.3043e-03],
        [2.5280e-03, 2.7676e-03, 4.3056e-03, 2.4223e-03, 1.6394e-03, 1.5180e-03,
         1.2183e-04, 6.7089e-03, 4.4837e-03, 2.9359e-03],
        [2.7592e-03, 3.0532e-03, 4.7543e-03, 2.7997e-03, 1.8287e-03, 1.7097e-03,
         1.5163e-04, 7.3860e-03, 5.0027e-03, 3.2947e-03],
        [1.6549e-03, 1.9262

In [6]:
norm_diff_w1_basic = torch.linalg.matrix_norm(grad_w1_numer_basic-grad_w1_theory_basic)
norm_diff_w2_basic = torch.linalg.matrix_norm(grad_w2_numer_basic-grad_w2_theory_basic)

print("The matrix norm of the difference between theoretical and numerical solutions of W1:", norm_diff_w1_basic.item())
print("The matrix norm of the difference between theoretical and numerical solutions of W2:", norm_diff_w2_basic.item())

The matrix norm of the difference between theoretical and numerical solutions of W1: 0.030822651460766792
The matrix norm of the difference between theoretical and numerical solutions of W2: 0.007077677641063929


In [7]:
# define loss function in terms of W1 and W2
def loss_func_W1_probs(W1):
    z = (mask_dropping_probs(prob, m, n)*X) @ W1.T @ W2.T - X
    return sum(sum(z*z)) / m / n

def loss_func_W2_probs(W2):
    z = (mask_dropping_probs(prob, m, n)*X) @ W1.T @ W2.T - X
    return sum(sum(z*z)) / m / n

In [8]:
# mask_dropping_probs
prob_list = torch.arange(1/(n+1), 1, 1/(n+1))
mean_m_probs = prob_list.repeat(m, 1)
square_m_probs = prob_list.view(n, 1) @ prob_list.view(1, n)
square_m_probs = square_m_probs.fill_diagonal_(0) + torch.diag(prob_list)

grad_w1_theory_probs = W2.T @ (W2@W1@(square_m_probs*(X.T@X)) - X.T@(mean_m_probs*X)) * (2/m/n)
grad_w2_theory_probs = (W2@W1@(square_m_probs*(X.T@X))-X.T@(mean_m_probs*X)) @ W1.T * (2/m/n)

# Initialising sampling
N = 10000
grad_w1_numer_probs = 0
grad_w2_numer_probs = 0
# Sampling process
for i in range(N):
    grad_w1_numer_probs += torch.autograd.functional.jacobian(loss_func_W1_probs, W1)
    grad_w2_numer_probs += torch.autograd.functional.jacobian(loss_func_W2_probs, W2)
grad_w1_numer_probs /= N
grad_w2_numer_probs /= N

abs(grad_w1_theory_probs-grad_w1_numer_probs)

tensor([[2.4490, 1.9657, 1.5445, 1.4475, 0.9734, 0.5408, 0.2401, 0.1141, 0.2931,
         0.6345],
        [2.5689, 2.0614, 1.6211, 1.5162, 1.0202, 0.5629, 0.2463, 0.1265, 0.3145,
         0.6730],
        [1.6108, 1.2885, 1.0163, 0.9511, 0.6412, 0.3554, 0.1584, 0.0754, 0.1920,
         0.4172],
        [2.0577, 1.6537, 1.3027, 1.2157, 0.8252, 0.4593, 0.2100, 0.0878, 0.2403,
         0.5207],
        [1.8875, 1.5133, 1.1894, 1.1077, 0.7513, 0.4151, 0.1866, 0.0877, 0.2264,
         0.4827],
        [2.3742, 1.8979, 1.4904, 1.3955, 0.9372, 0.5179, 0.2269, 0.1187, 0.2902,
         0.6219],
        [2.6407, 2.1255, 1.6657, 1.5641, 1.0588, 0.5913, 0.2722, 0.1079, 0.3017,
         0.6642],
        [1.5969, 1.3003, 1.0181, 0.9544, 0.6465, 0.3620, 0.1662, 0.0636, 0.1816,
         0.4054],
        [2.0238, 1.6330, 1.2797, 1.1944, 0.8079, 0.4462, 0.1985, 0.0950, 0.2446,
         0.5229],
        [1.8703, 1.5001, 1.1764, 1.0996, 0.7392, 0.4076, 0.1781, 0.0941, 0.2302,
         0.4921]], grad_fn=<

In [9]:
norm_diff_w1_probs = torch.linalg.matrix_norm(grad_w1_numer_probs-grad_w1_theory_probs)
norm_diff_w2_probs = torch.linalg.matrix_norm(grad_w2_numer_probs-grad_w2_theory_probs)

print("The matrix norm of the difference between theoretical and numerical solutions of W1:", norm_diff_w1_probs.item())
print("The matrix norm of the difference between theoretical and numerical solutions of W2:", norm_diff_w2_probs.item())

The matrix norm of the difference between theoretical and numerical solutions of W1: 11.097588539123535
The matrix norm of the difference between theoretical and numerical solutions of W2: 6.249958038330078


In [10]:
# define loss function in terms of W1 and W2
def loss_func_W1_patches(W1):
    z = (mask_patches(prob, patch_size, m, n)*X) @ W1.T @ W2.T - X
    return sum(sum(z*z)) / m / n

def loss_func_W2_patches(W2):
    z = (mask_patches(prob, patch_size, m, n)*X) @ W1.T @ W2.T - X
    return sum(sum(z*z)) / m / n

In [11]:
# mask_patches
patch_size = 2
mean_m_patches = torch.ones(m, n) * prob
square_m_patches = torch.ones(n, n) * prob**2
for i in range(n):
    place_val = i // patch_size
    square_m_patches[place_val:place_val+patch_size, place_val:place_val+patch_size] = torch.ones(patch_size, patch_size) * prob

grad_w1_theory_patches = W2.T @ (W2@W1@(square_m_patches*(X.T@X)) - X.T@(mean_m_patches*X)) * (2/m/n)
grad_w2_theory_patches = (W2@W1@(square_m_patches*(X.T@X))-X.T@(mean_m_patches*X)) @ W1.T * (2/m/n)

# Initialising sampling
N = 10000
grad_w1_numer_patches = 0
grad_w2_numer_patches = 0
# Sampling process
for i in range(N):
    grad_w1_numer_patches += torch.autograd.functional.jacobian(loss_func_W1_patches, W1)
    grad_w2_numer_patches += torch.autograd.functional.jacobian(loss_func_W2_patches, W2)
grad_w1_numer_patches /= N
grad_w2_numer_patches /= N

abs(grad_w1_theory_patches-grad_w1_numer_patches)

tensor([[0.0014, 0.1879, 0.0912, 0.2357, 0.2120, 0.0069, 0.4283, 0.4298, 0.3227,
         0.3464],
        [0.0015, 0.2015, 0.0916, 0.2490, 0.2223, 0.0073, 0.4491, 0.4493, 0.3380,
         0.3626],
        [0.0009, 0.1259, 0.0574, 0.1542, 0.1416, 0.0045, 0.2828, 0.2835, 0.2115,
         0.2266],
        [0.0013, 0.1616, 0.0747, 0.2093, 0.1675, 0.0059, 0.3497, 0.3514, 0.2691,
         0.2879],
        [0.0012, 0.1486, 0.0675, 0.1915, 0.1483, 0.0054, 0.3197, 0.3217, 0.2464,
         0.2651],
        [0.0013, 0.1837, 0.0843, 0.2254, 0.2037, 0.0067, 0.4179, 0.4207, 0.3141,
         0.3380],
        [0.0016, 0.2035, 0.1063, 0.2615, 0.2164, 0.0075, 0.4460, 0.4491, 0.3418,
         0.3696],
        [0.0010, 0.1253, 0.0690, 0.1621, 0.1343, 0.0046, 0.2729, 0.2729, 0.2101,
         0.2247],
        [0.0013, 0.1586, 0.0779, 0.2066, 0.1607, 0.0058, 0.3452, 0.3458, 0.2662,
         0.2859],
        [0.0011, 0.1453, 0.0693, 0.1808, 0.1565, 0.0053, 0.3297, 0.3317, 0.2486,
         0.2672]], grad_fn=<

In [12]:
norm_diff_w1_patches = torch.linalg.matrix_norm(grad_w1_numer_patches-grad_w1_theory_patches)
norm_diff_w2_patches = torch.linalg.matrix_norm(grad_w2_numer_patches-grad_w2_theory_patches)

print("The matrix norm of the difference between theoretical and numerical solutions of W1:", norm_diff_w1_patches.item())
print("The matrix norm of the difference between theoretical and numerical solutions of W2:", norm_diff_w2_patches.item())

The matrix norm of the difference between theoretical and numerical solutions of W1: 2.3551125526428223
The matrix norm of the difference between theoretical and numerical solutions of W2: 1.1436023712158203
